# Learning Objectives

In this notebook, you will craft sophisticated ETL jobs that interface with a variety of common data sources, such as 
- REST APIs (HTTP endpoints)
- RDBMS
- Hive tables (managed tables)
- Various file formats (csv, json, parquet, etc.)

d

# Interview Questions

As you progress through the practice, attempt to answer the following questions:

## Columnar File
- What is a columnar file format and what advantages does it offer?
- Why is Parquet frequently used with Spark and how does it function?
- How do you read/write data from/to a Parquet file using a DataFrame?

## Partitions
- How do you save data to a file system by partitions? (Hint: Provide the code)
- How and why can partitions reduce query execution time? (Hint: Give an example)

## JDBC and RDBMS
- How do you load data from an RDBMS into Spark? (Hint: Discuss the steps and JDBC)

## REST API and HTTP Requests
- How can Spark be used to fetch data from a REST API? (Hint: Discuss making API requests)

## ETL Job One: Parquet file
### Extract
Extract data from the managed tables (e.g. `bookings_csv`, `members_csv`, and `facilities_csv`)

### Transform
Data transformation requirements https://pgexercises.com/questions/aggregates/fachoursbymonth.html

### Load
Load data into a parquet file

### What is Parquet? 

Columnar files are an important technique for optimizing Spark queries. Additionally, they are often tested in interviews.
- https://www.youtube.com/watch?v=KLFadWdomyI
- https://www.databricks.com/glossary/what-is-parquet

In [0]:
# Write your solution here
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws
from pyspark.sql.window import Window
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("ETL").getOrCreate()

members_df = spark.read.format("csv").option("header", "true").load("members.csv")
bookings_df = spark.read.format("csv").option("header", "true").load("bookings.csv")
facilities_df = spark.read.format("csv").option("header", "true").load("facilities.csv")

members_df.createOrReplaceTempView("members")
bookings_df.createOrReplaceTempView("bookings")
facilities_df.createOrReplaceTempView("facilities")

result_df = spark.sql("""
    SELECT DISTINCT
        concat_ws(' ', mems.firstname, mems.surname) as member,
        facs.name as facility
    FROM
        members mems
        INNER JOIN bookings bks ON mems.memid = bks.memid
        INNER JOIN facilities facs ON bks.facid = facs.facid
    WHERE
        facs.name IN ('Tennis Court 2', 'Tennis Court 1')
    ORDER BY
        member, facility
""")

result_df.write.mode("overwrite").parquet("output.parquet")

spark.stop()


## ETL Job Two: Partitions

### Extract
Extract data from the managed tables (e.g. `bookings_csv`, `members_csv`, and `facilities_csv`)

### Transform
Transform the data https://pgexercises.com/questions/joins/threejoin.html

### Load
Partition the result data by facility column and then save to `threejoin_delta` managed table. Additionally, they are often tested in interviews.

hint: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.partitionBy.html

What are paritions? 

Partitions are an important technique to optimize Spark queries
- https://www.youtube.com/watch?v=hvF7tY2-L3U&t=268s

In [0]:
# Write your solution here

from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("ETL").getOrCreate()

members_df = spark.read.format("delta").load("/path/to/members_delta")
bookings_df = spark.read.format("delta").load("/path/to/bookings_delta")
facilities_df = spark.read.format("delta").load("/path/to/facilities_delta")

result_df = members_df.join(
    bookings_df, "memid"
).join(
    facilities_df, "facid"
).filter(
    facilities_df.name.isin("Tennis Court 2", "Tennis Court 1")
).select(
    concat_ws(" ", members_df.firstname, members_df.surname).alias("member"),
    facilities_df.name.alias("facility")
).distinct().orderBy("member", "facility")

result_df.write.format("delta").mode("overwrite").save("/path/to/threejoin_delta")

spark.stop()

## ETL Job Three: HTTP Requests

### Extract
Extract daily stock price data price from the following companies, Google, Apple, Microsoft, and Tesla. 

Data Source
- API: https://rapidapi.com/alphavantage/api/alpha-vantage
- Endpoint: GET `TIME_SERIES_DAILY`

Sample HTTP request

```
curl --request GET \
	--url 'https://alpha-vantage.p.rapidapi.com/query?function=TIME_SERIES_DAILY&symbol=TSLA&outputsize=compact&datatype=json' \
	--header 'X-RapidAPI-Host: alpha-vantage.p.rapidapi.com' \
	--header 'X-RapidAPI-Key: [YOUR_KEY]'

```

Sample Python HTTP request

```
import requests

url = "https://alpha-vantage.p.rapidapi.com/query"

querystring = {
    "function":"TIME_SERIES_DAILY",
    "symbol":"IBM",
    "datatype":"json",
    "outputsize":"compact"
}

headers = {
    "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com",
    "X-RapidAPI-Key": "[YOUR_KEY]"
}

response = requests.get(url, headers=headers, params=querystring)

data = response.json()

# Now 'data' contains the daily time series data for "IBM"
```

### Transform
Find **weekly** max closing price for each company.

hints: 
  - Use a `for-loop` to get stock data for each company
  - Use the spark `union` operation to concat all data into one DF
  - create a new `week` column from the data column
  - use `group by` to calcualte max closing price

### Load
- Partition `DF` by company
- Load the DF in to a managed table called, `max_closing_price_weekly`

In [0]:
# Write your solution here

import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, max
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("StockPriceETL").getOrCreate()

companies = [
    {"name": "Google", "symbol": "GOOGL"},
    {"name": "Apple", "symbol": "AAPL"},
    {"name": "Microsoft", "symbol": "MSFT"},
    {"name": "Tesla", "symbol": "TSLA"}
]

url = "https://alpha-vantage.p.rapidapi.com/query"
headers = {
    "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com",
    "X-RapidAPI-Key": "[YOUR_KEY]"
}

stock_data_df = spark.createDataFrame([], ["date", "company", "symbol", "closing_price"])

for company_info in companies:
    querystring = {
        "function": "TIME_SERIES_DAILY",
        "symbol": company_info["symbol"],
        "datatype": "json",
        "outputsize": "compact"
    }

    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()

    daily_data = [(date, company_info["name"], company_info["symbol"], float(daily_data["4. close"]))
                  for date, daily_data in data["Time Series (Daily)"].items()]

    daily_data_df = spark.createDataFrame(daily_data, ["date", "company", "symbol", "closing_price"])
    stock_data_df = stock_data_df.union(daily_data_df)

stock_data_df = stock_data_df.withColumn("date", to_date(stock_data_df["date"], "yyyy-MM-dd"))
stock_data_df = stock_data_df.withColumn("week", F.weekofyear(stock_data_df["date"]))

window_spec = Window.partitionBy("company", "week")
max_closing_price_df = stock_data_df.withColumn("max_closing_price", max("closing_price").over(window_spec))

max_closing_price_df = max_closing_price_df.drop("date", "closing_price", "week", "symbol")
max_closing_price_df = max_closing_price_df.dropDuplicates()

max_closing_price_df.write.mode("overwrite").saveAsTable("max_closing_price_weekly")

spark.stop()

## ETL Job Four: RDBMS


### Extract
Extract RNA data from a public PostgreSQL database.

- https://rnacentral.org/help/public-database
- Extract 100 RNA records from the `rna` table (hint: use `limit` in your sql)
- hint: use `spark.read.jdbc` https://docs.databricks.com/external-data/jdbc.html

### Transform
We want to load the data as it so there is no transformation required.


### Load
Load the DF in to a managed table called, `rna_100_records`

In [0]:
# Write your solution here

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RNADataETL").getOrCreate()

jdbc_url = "jdbc:postgresql://your_postgresql_host:5432/your_database"
properties = {
    "user": "your_username",
    "password": "your_password",
    "driver": "org.postgresql.Driver"
}

sql_query = "SELECT * FROM rna LIMIT 100"

rna_df = spark.read.jdbc(url=jdbc_url, table=sql_query, properties=properties)

rna_df.write.mode("overwrite").saveAsTable("rna_100_records")

spark.stop()